In [ ]:
# pd.set_option('display.max_columns', None)
# pd.set_option('display.max_rows', None)

#読み込み

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!pip install xgboost
!pip install optuna
!pip install lightgbm
!pip install catboost

In [ ]:
# データ操作
import numpy as np
import pandas as pd

# データ可視化
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

#日付の扱い
import datetime
from datetime import timedelta

#正規化とか
import scipy.stats
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler

# 交差検証
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.model_selection import KFold, cross_val_score, cross_validate

#誤差確認
from sklearn.metrics import make_scorer
from sklearn.metrics import mean_squared_error

#重回帰
from sklearn.linear_model import LinearRegression

#ランダムフォレスト
from sklearn import datasets
from sklearn.ensemble import RandomForestRegressor

#DNN
from keras.models import Sequential
from keras.layers import Input, Dense, Dropout, BatchNormalization
from keras.wrappers.scikit_learn import KerasRegressor
from sklearn.pipeline import Pipeline

#XGBoost
import xgboost as xgb

#LightGBM
import lightgbm as lgb

#CatBoost
from catboost import CatBoost
from catboost import Pool

#パラメータチューニング
import optuna

#警告無視
import warnings
warnings.filterwarnings('ignore')

In [ ]:
train=pd.read_csv("/content/drive/MyDrive/Colab Notebooks/SIGNATE/minpaku_price/train.csv")
test=pd.read_csv("/content/drive/MyDrive/Colab Notebooks/SIGNATE/minpaku_price/test.csv")

In [ ]:
# 定数の定義
SEED = 2019

In [ ]:
train.shape

In [ ]:
train.columns

In [ ]:
test.shape

In [ ]:
test.columns

#前処理

In [ ]:
#欠損値処理のために一旦統合
#TrainFlagという列を新たに作り、訓練データにはTrue、テストデータにはFalseとしておく。
train["TrainFlag"] = True
test["TrainFlag"] = False

#訓練データとテストデータを結合する
alldata = train.append(test)

#目的変数を先頭コラムに
cols=alldata.columns.tolist()
cols=cols[-2:]+cols[:-2]

alldata =  alldata[cols] 
alldata

In [ ]:
alldata.tail()

In [ ]:
alldata.info()

In [ ]:
alldata.describe()

In [ ]:
alldata.describe(include=['O'])

In [ ]:
#欠損値がある変数だけが降順に並び替え
alldata.isnull().sum()[alldata.isnull().sum()>0].sort_values(ascending = False)

###host_response_rate 100%とかのobjectを数字にする

In [ ]:
df_res = alldata['host_response_rate'].str.extract(r"(\d+)([^\d]+)")
df_res = df_res.rename(columns={0: 'host_response_rate_num'})
df_res = df_res.drop(1,axis=1)
df_res.host_response_rate_num=df_res.host_response_rate_num.astype(float)
df_res

In [ ]:
df_res.shape

In [ ]:
alldata_res=pd.concat([alldata, df_res],axis=1)
alldata_res=alldata_res.drop(["host_response_rate"],axis=1)
alldata_res.head()

In [ ]:
alldata_res.shape

In [ ]:
plt.scatter(alldata_res.host_response_rate_num,alldata_res.y)

In [ ]:
#中央値で穴埋め
alldata_res.host_response_rate_num = alldata_res.host_response_rate_num.fillna(alldata_res.host_response_rate_num.median())

#欠損値がある変数だけが降順に並び替え
alldata_res.isnull().sum()[alldata_res.isnull().sum()>0].sort_values(ascending = False)

In [ ]:
alldata_res.loc[alldata_res.host_response_rate_num ==0,"response_rate_group"]="0"
alldata_res.loc[(alldata_res.host_response_rate_num >0) & (alldata_res.host_response_rate_num <50),"response_rate_group"]="050"
alldata_res.loc[(alldata_res.host_response_rate_num >=50) & (alldata_res.host_response_rate_num <75),"response_rate_group"]="5075"
alldata_res.loc[(alldata_res.host_response_rate_num >=75) & (alldata_res.host_response_rate_num <97) ,"response_rate_group"]="7597"
alldata_res.loc[alldata_res.host_response_rate_num >=97 ,"response_rate_group"]="97100"

alldata_res.drop(["host_response_rate_num"],axis=1)

alldata_res.head()

In [ ]:
alldata_res.shape

In [ ]:
plt.scatter(alldata_res.response_rate_group,alldata_res.y)

###review_scores_rating

In [ ]:
sns.distplot(alldata_res["review_scores_rating"])


In [ ]:
sns.distplot(np.log(alldata_res["review_scores_rating"]))

In [ ]:
plt.scatter(alldata_res.review_scores_rating,alldata_res.y)

In [ ]:
#中央値で穴埋め
alldata_res.review_scores_rating = alldata_res.review_scores_rating.fillna(alldata_res.review_scores_rating.median())

#欠損値がある変数だけが降順に並び替え
alldata_res.isnull().sum()[alldata_res.isnull().sum()>0].sort_values(ascending = False)

In [ ]:
# #特徴量の追加
#あまり効果無さそう
#alldata_res['hasreview'] = alldata_res['review_scores_rating'].apply(lambda x: 1 if x > 0 else 0)

#plt.scatter(alldata_res.hasreview,alldata_res.y)
#print(alldata_res[alldata_res.hasreview==1].y.mean())#reviewあり
#print(alldata_res[alldata_res.hasreview==0].y.mean())#reviewなし

#alldata_res=alldata_res.drop(["hasreview"],axis=1)
#alldata_res.head()

In [ ]:
#sns.swarmplot(alldata_res.city,alldata_res.y,data=alldata_res)

In [ ]:
#pd.plotting.scatter_matrix(alldata_res)

In [ ]:
#alldata_res.thumbnail_url
# #特徴量の追加
#あまり効果無さそう
alldata_res['hasthumbnail'] = alldata_res['thumbnail_url'].apply(lambda x: 1 if pd.isna(x) != True else 0)
alldata_res.tail()
#plt.scatter(alldata_res.hasreview,alldata_res.y)

サムネイル持っている方が高い，という仮説違ってそう

In [ ]:
plt.scatter(alldata_res.hasthumbnail,alldata_res.y)
print(alldata_res[alldata_res.hasthumbnail==1].y.mean())
print(alldata_res[alldata_res.hasthumbnail==0].y.mean())

In [ ]:
alldata_res=alldata_res.drop(["thumbnail_url",'hasthumbnail'],axis=1)
alldata_res.head()

In [ ]:
#欠損値がある変数だけが降順に並び替え
alldata_res.isnull().sum()[alldata_res.isnull().sum()>0].sort_values(ascending = False)

In [ ]:
alldata_res.shape

###neighbourhoodの中身

In [ ]:
n_count=alldata_res["neighbourhood"].value_counts()
n_count

In [ ]:
alldata_res["neighbourhood"].values

In [ ]:
#df = pd.DataFrame(columns=alldata_res["neighbourhood"].unique()[1:])
col = alldata_res["neighbourhood"].unique()[1:]
df = pd.DataFrame(index=[], columns=col)
record = pd.Series(['hoge']*619, index=df.columns)
df = df.append(record, ignore_index=True)


for i in alldata_res["neighbourhood"].unique()[1:]:
  ave=alldata_res[alldata_res.neighbourhood==i].y.mean()
  if ave>=300 and n_count[i]>=10:
    df[i]=ave
  else:
    df=df.drop(i,axis=1)

  #df=df.assign(i=ave)
  #print(ave,i)
df

In [ ]:
for i in df.columns:
  print(n_count[i],i)

In [ ]:
#plt.figuresize(figsize=(10,10)
df.plot.barh(figsize=(10,10))

In [ ]:
#alldata_res["neighbourhood"]について，df.columnsととothersにする
alldata_res.loc[~alldata_res.neighbourhood.isin(df.columns),"neighbourhood"] ='others'
alldata_res.loc[alldata_res.neighbourhood.isnull()==True,"neighbourhood"] ='others'
alldata_res

In [ ]:
#欠損値がある変数だけが降順に並び替え
alldata_res.isnull().sum()[alldata_res.isnull().sum()>0].sort_values(ascending = False)

###bedrooms

In [ ]:
plt.hist(alldata_res.bedrooms)

bedroom数は中央値で穴埋め

In [ ]:
alldata_res.bedrooms=alldata_res.bedrooms.fillna(alldata_res.bedrooms.median())
alldata_res

In [ ]:
plt.scatter(alldata_res.bedrooms,alldata_res.y)

In [ ]:
plt.scatter(alldata_res.bedrooms,alldata_res.beds)
plt.xlabel("bedrooms")
plt.ylabel("beds")

In [ ]:
plt.scatter(alldata_res.beds,alldata_res.y)

In [ ]:
plt.scatter(alldata_res.beds/alldata_res.bedrooms,alldata_res.y)
#alldata_res.beds/alldata_res
alldata_res["alldata_res.beds/alldata_res.bedrooms"]=alldata_res.beds/alldata_res.bedrooms
alldata_res["alldata_res.beds/alldata_res.bedrooms"].median()


In [ ]:
alldata_res=alldata_res.drop('alldata_res.beds/alldata_res.bedrooms', axis=1)
alldata_res.head(1)

In [ ]:
alldata_res.shape

beds=bedroomsで穴埋め

In [ ]:
plt.scatter(alldata_res.accommodates,alldata_res.bedrooms)

In [ ]:
plt.scatter(alldata_res.accommodates,alldata_res.y)

bedroom数がbed数を超えているものは修正

In [ ]:
alldata_res[alldata_res.bedrooms==10]
#31081のデータ誤ってそう

In [ ]:
#alldata_res.loc[alldata_res.bedrooms > alldata_res.beds , "bedrooms"] = alldata_res.loc[alldata_res.bedrooms > alldata_res.beds , "beds"]
#alldata_res

In [ ]:
#alldata_res[alldata_res.bedrooms==10]

In [ ]:
plt.scatter(alldata_res.bedrooms,alldata_res.beds)
plt.xlabel("bedrooms")
plt.ylabel("beds")

In [ ]:
plt.hist(alldata_res.bedrooms)

In [ ]:
plt.scatter(alldata_res.accommodates,alldata_res.beds)

In [ ]:
alldata_res[alldata_res.accommodates<alldata_res.beds]

In [ ]:
alldata_res.shape

In [ ]:
model_lr = LinearRegression()
alldata_drop=alldata_res
alldata_drop=alldata_drop.dropna(subset=['beds'])
alldata_drop2=alldata_drop
alldata_drop2=alldata_drop2.drop(alldata_drop2.accommodates=="Shared room")
x=alldata_drop2.accommodates.values.reshape(-1, 1)
y=alldata_drop2.beds.values.reshape(-1, 1)
model_lr.fit(x, y)
plt.plot(x, y, 'o')
plt.plot(x, model_lr.predict(x), linestyle="solid")
plt.show()

print('モデル関数の回帰変数 w1: %.3f' %model_lr.coef_)
print('モデル関数の切片 w2: %.3f' %model_lr.intercept_)
print('y= %.3fx + %.3f' % (model_lr.coef_ , model_lr.intercept_))
print('決定係数 R^2： ', model_lr.score(x, y))

In [ ]:
alldata_res.shape

In [ ]:
alldata_drop[(alldata_drop.beds>=5) & (alldata_drop.accommodates==1)]

In [ ]:
plt.scatter(alldata_drop.room_type,alldata_drop.y)

In [ ]:
alldata_drop.groupby("room_type").beds.mean()

In [ ]:
alldata_res.shape

In [ ]:
#2行1列のaxesを生成
fig,ax=plt.subplots(3,1,figsize=(15,15))

df_E  =alldata_drop[alldata_drop["room_type"]=="Entire home/apt"][["accommodates","beds"]]
df_P=alldata_drop[alldata_drop["room_type"]=="Private room"][["accommodates","beds"]]
df_S  =alldata_drop[alldata_drop["room_type"]=="Shared room"][["accommodates","beds"]]


ax[0].scatter(df_E["accommodates"],df_E["beds"],label="E",color="blue")
ax[1].scatter(df_P["accommodates"],df_P["beds"],label="P",color="red")
ax[2].scatter(df_S["accommodates"],df_S["beds"],label="S",color="green")

for i in range(2):
    ax[i].set_xlabel("accommodates")
    ax[i].set_ylabel("beds")
    ax[i].legend()

In [ ]:
#if alldata_res["room_type"]=='Entire home/apt' or alldata_res["room_type"]=='Private room':

#for column in alldata.columns:
    # dtypeがobjectの場合、文字列の変数
#    if alldata[column].dtype=='O':
#        alldata[column] = alldata[column].fillna(alldata[column].mode().iloc[0])
    # dtypeがint , floatの場合、数字の変数
#    else:
#        alldata[column] = alldata[column].fillna(alldata[column].median())

In [ ]:
alldata_res.beds=alldata_res.beds.fillna(alldata_res.bedrooms)
#欠損値がある変数だけが降順に並び替え
alldata_res.isnull().sum()[alldata_res.isnull().sum()>0].sort_values(ascending = False)

In [ ]:
alldata_res.shape

In [ ]:
#alldata_res.groupby("host_has_profile_pic").y.mean()
plt.hist(alldata_res[alldata_res["host_has_profile_pic"]=="t"].y,bins=100)
plt.show()
plt.hist(alldata_res[alldata_res["host_has_profile_pic"]=="f"].y,bins=100)
plt.show()

In [ ]:
alldata_res.groupby("host_identity_verified").y.mean()
plt.hist(alldata_res[alldata_res["host_identity_verified"]=="t"].y,bins=100)
plt.show()
plt.hist(alldata_res[alldata_res["host_identity_verified"]=="f"].y,bins=100)
plt.show()

host_identity_verifiedとhost_has_profile_picは値段に関係なさそうなので削除

In [ ]:
alldata_res=alldata_res.drop(['host_identity_verified','host_has_profile_pic'], axis=1)
alldata_res.head(1)

In [ ]:
alldata_res.shape

In [ ]:
#欠損値がある変数だけが降順に並び替え
alldata_res.isnull().sum()[alldata_res.isnull().sum()>0].sort_values(ascending = False)

In [ ]:
alldata_drop=alldata_drop.dropna(subset=['first_review', 'last_review'])


alldata_drop["first_review_datetype"]=pd.to_datetime(alldata_drop["first_review"])
alldata_drop["last_review_datetype"]=pd.to_datetime(alldata_drop["last_review"])
alldata_drop["host_since_datetype"]=pd.to_datetime(alldata_drop["host_since"])

alldata_drop["DateDelta_L-F"]=alldata_drop["last_review_datetype"] - alldata_drop["first_review_datetype"]
alldata_drop["Days_Int_L-F"] = (alldata_drop["DateDelta_L-F"] / timedelta(days=1))
alldata_drop.head(1)

print(alldata_drop["first_review_datetype"].mean())
print(alldata_drop["last_review_datetype"].mean())
print(alldata_drop["host_since_datetype"].mean())

In [ ]:
plt.scatter(alldata_drop["Days_Int_L-F"],   alldata_drop.y)

In [ ]:
alldata_drop["DateDelta_F-S"]=  alldata_drop["first_review_datetype"] -alldata_drop["host_since_datetype"]
alldata_drop["Days_Int_F-S"] = (alldata_drop["DateDelta_F-S"] / timedelta(days=1))
plt.scatter(alldata_drop["Days_Int_F-S"],   alldata_drop.y)

In [ ]:
alldata_drop["DateDelta_L-S"]=  alldata_drop["last_review_datetype"] -alldata_drop["host_since_datetype"]
alldata_drop["Days_Int_L-S"] = (alldata_drop["DateDelta_L-S"] / timedelta(days=1))
plt.scatter(alldata_drop["Days_Int_L-S"],   alldata_drop.y)

最終review日-oldest review日が値段に関係ありそう

In [ ]:
alldata_res=alldata_res.dropna(subset=['host_since'])

print(alldata_res.shape)

alldata_res["first_review"]=alldata_res["first_review"].fillna("2016-01-14")#average
alldata_res["last_review"]=alldata_res["last_review"].fillna("2017-03-14")#average

print(alldata_res.shape)

alldata_res["first_review_datetype"]=pd.to_datetime(alldata_res["first_review"])
alldata_res["last_review_datetype"]=pd.to_datetime(alldata_res["last_review"])

print(alldata_res.shape)

alldata_res["DateDelta_L-F"]=alldata_res["last_review_datetype"] - alldata_res["first_review_datetype"]
alldata_res["Days_Int_L-F"] = (alldata_res["DateDelta_L-F"] / timedelta(days=1))

print(alldata_res.shape)

alldata_res=alldata_res.drop(['DateDelta_L-F','first_review_datetype','last_review_datetype'],axis=1)

#print(alldata_drop["first_review_datetype"].mean())
#print(alldata_drop["last_review_datetype"].mean())

In [ ]:
alldata_res.shape

In [ ]:
alldata_res.head(1)

In [ ]:
alldata_res.shape

###amenities

In [ ]:
type(alldata_res.amenities[0])

In [ ]:
alldata_drop.amenities=alldata_drop.amenities.str.strip('{')
alldata_drop.amenities=alldata_drop.amenities.str.strip('}')
#alldata_drop.amenities=alldata_drop.amenities.str.strip('V')
#alldata_drop.head()
alldata_drop.amenities[0]
#for i in alldata_drop.amenities.index:
#  str_ame=alldata_drop.amenities[i].astype(str).strip('"')
#  str_ame
#print(alldata_drop.amenities[0].astype(str).strip('"'))

In [ ]:
alldata_amenities=alldata_drop.amenities.str.split(',', expand=True)
alldata_amenities

In [ ]:
amenities_set=set()
#set(alldata_amenities.iloc[i].values)
for i in alldata_amenities.index:
  amenities_set.update(tuple(alldata_amenities.iloc[i].values))

amenities_list=[]
for word in amenities_set:
  if type(word)== str:
    word=word.strip('"')
    #print(word)
    amenities_list.append(word)
print(amenities_list,len(amenities_list))
  # word2=word.str.strip('"')
  # amenities_set2.update(word2)

In [ ]:
#for i in alldata_drop.index:
#if alldata_drop.amenities.str.contains("TV") == True:
#  alldata_drop["hasTV"]=True
#else:
#  alldata_drop["hasTV"]=False
#alldata_drop

#alldata_drop["hasTV"]=alldata_drop.amenities.str.contains("TV")
#alldata_drop

In [ ]:
#amenities_list=list(amenities_set)
for i in amenities_list:
  alldata_drop["has_"+i]=alldata_drop.amenities.str.contains(i)
alldata_drop

In [ ]:
alldata_res.shape

本当のdataframeに適用

In [ ]:
alldata_res.amenities=alldata_res.amenities.str.strip('{')
alldata_res.amenities=alldata_res.amenities.str.strip('}')

alldata_amenities_res=alldata_res.amenities.str.split(',', expand=True)
amenities_set_res=set()

for i in alldata_amenities_res.index:
  amenities_set_res.update(tuple(alldata_amenities_res.iloc[i].values))

amenities_list=[]
for word in amenities_set_res:
  if type(word)== str:
    word=word.strip('"')
    #print(word)
    amenities_list.append(word)
print(amenities_list,len(amenities_list))
  # word2=word.str.strip('"')
  # amenities_set2.update(word2)
for i in amenities_list:
  alldata_res["has_"+i]=alldata_res.amenities.str.contains(i)

In [ ]:
alldata_res.shape

In [ ]:
alldata_res=alldata_res.drop(["amenities","has_","has_translation missing: en.hosting_amenity_49"],axis=1)
alldata_res.head()

In [ ]:
#欠損値がある変数だけが降順に並び替え
alldata_res.isnull().sum()[alldata_res.isnull().sum()>0].sort_values(ascending = False)

###zipcode

In [ ]:
alldata_res.zipcode.value_counts

In [ ]:
alldata_drop=alldata_drop.dropna(subset=['zipcode'])
#欠損値がある変数だけが降順に並び替え
alldata_drop.isnull().sum()[alldata_drop.isnull().sum()>0].sort_values(ascending = False)

In [ ]:
g_zip5=[]
g_zip4=[]
g_zip3=[]
g_zip2=[]
for i in alldata_drop.index:
  g_zip5.append(alldata_drop.zipcode.values[i][0:5])#
  g_zip4.append(alldata_drop.zipcode.values[i][0:4])#
  g_zip3.append(alldata_drop.zipcode.values[i][0:3])#
  g_zip2.append(alldata_drop.zipcode.values[i][0:2])#
  #print(g)
alldata_drop["Global_zip5"]=g_zip5
alldata_drop["Global_zip4"]=g_zip4
alldata_drop["Global_zip3"]=g_zip3
alldata_drop["Global_zip2"]=g_zip2
# alldata_drop.head()

In [ ]:
plt.scatter(alldata_drop.Global_zip5, alldata_drop.y)
plt.show()
plt.scatter(alldata_drop.Global_zip4, alldata_drop.y)
plt.show()
plt.scatter(alldata_drop.Global_zip3, alldata_drop.y)
plt.show()
plt.scatter(alldata_drop.Global_zip2, alldata_drop.y)
plt.show()

In [ ]:
alldata_drop.Global_zip4.unique()

In [ ]:
#pd.set_option('display.max_rows', None)
alldata_drop.groupby("Global_zip4")["Global_zip4"].count()

In [ ]:
#zipcode関係ありそう 4桁でいいかな

In [ ]:
alldata_res.city.unique()

LA:90台
DC:20台
NY:10台
SF:94台
Chicago:60台
Boston:02台

In [ ]:
alldata_drop.info()

In [ ]:
# alldata_res.loc[(alldata_res.city =='LA') & (alldata_res.zipcode.isnull()==True),"zipcode"]="9080"
# alldata_resloc[(alldata_res.city=='DC') &  (alldata_res.zipcode.isnull()==True)].zipcode="2000"
# alldata_res.loc[(alldata_res.city=='NYC') & (alldata_res.zipcode.isnull()==True),"zipcode"]="1001"
# alldata_resloc[(alldata_res.city=='SF') &  (alldata_res.zipcode.isnull()==True)].zipcode="9411"
# alldata_resloc[(alldata_res.city=='Chicago') & (alldata_res.zipcode.isnull()==True)].zipcode="6061"
# alldata_resloc[(alldata_res.city=='Boston') & (alldata_res.zipcode.isnull()==True)].zipcode="0211"


zipcode_list=["9080","2000", "1001", "9411", "6061", "0211"]

for i in range(len(alldata_res.city.unique())):
  alldata_res.loc[(alldata_res.city == alldata_res.city.unique()[i]) & (alldata_res.zipcode.isnull()==True),"zipcode"]=zipcode_list[i]

g_zip4_res=[]
for i in alldata_res.index:
  g_zip4_res.append(alldata_res.zipcode.values[i][0:4])#

alldata_res["Global_zip4"]=g_zip4_res
alldata_res=alldata_res.drop(['zipcode'], axis=1)

alldata_res.head()

In [ ]:
# alldata_res.zipcode.isnull()

###bathrooms

In [ ]:
# sns.jointplot(alldata_res.accommodates, alldata_res.bathrooms, kind="hex")

In [ ]:
plt.hist(alldata_res.bathrooms)

bathroomの欠損値は1で穴埋め

In [ ]:
alldata_res.bathrooms=alldata_res.bathrooms.fillna(alldata_res.bathrooms.mean())

In [ ]:
#欠損値がある変数だけが降順に並び替え
alldata_res.isnull().sum()[alldata_res.isnull().sum()>0].sort_values(ascending = False)

###description,nameは保留

###消したり整理

In [ ]:
alldata_res.head(1)

In [ ]:
alldata_res=alldata_res.drop(['id','first_review','host_since','last_review', 'description', 'name'], axis=1)

In [ ]:
alldata_res.head()

###目的関数の分布

In [ ]:
sns.distplot(alldata_res.y)

In [ ]:
#対数変換後
sns.distplot(np.log(alldata_res.y))

In [ ]:
#log_y= np.log(alldata_res.y)

###longitude

In [ ]:
plt.scatter(alldata_res.longitude, alldata_res.y)

In [ ]:
#longitude_list=[-120, -90, -80, -70]


alldata_res.loc[alldata_res.longitude <-110,"longitude_group"]="-120"
alldata_res.loc[(alldata_res.longitude >-90) & (alldata_res.longitude <-80),"longitude_group"]="-90"
alldata_res.loc[(alldata_res.longitude >-80) & (alldata_res.longitude <-72.5),"longitude_group"]="-80"
alldata_res.loc[alldata_res.longitude >=-72.5 ,"longitude_group"]="-70"

alldata_res.drop(["longitude"],axis=1)

alldata_res.head()

###latitude

In [ ]:
plt.scatter(alldata_res.latitude, alldata_res.y)

In [ ]:
#latitude_list=[34, 38, 41, 42]


alldata_res.loc[alldata_res.latitude <36,"latitude_group"]="34"
alldata_res.loc[(alldata_res.latitude >36) & (alldata_res.latitude <40),"latitude_group"]="38"
alldata_res.loc[(alldata_res.latitude >40) & (alldata_res.latitude <41.3),"latitude_group"]="41"
alldata_res.loc[alldata_res.latitude >=41.3 ,"latitude_group"]="42"

alldata_res.drop(["latitude"],axis=1)

alldata_res.head()

###cancellation_policy

In [ ]:
plt.scatter(alldata_res.cancellation_policy, alldata_res.y)

In [ ]:
# sns.jointplot("cancellation_policy", "y", data=alldata_res, kind="hex")
alldata_res.groupby("cancellation_policy").y.mean()

In [ ]:
alldata_res.groupby("cancellation_policy").y.median()

###instant_bookable

In [ ]:
alldata_res.groupby("instant_bookable").y.mean()

In [ ]:
plt.scatter(alldata_res.instant_bookable, alldata_res.y)

###number_of_reviews

In [ ]:
plt.scatter(alldata_res.number_of_reviews, alldata_res.y)

###response_rate

In [ ]:
plt.scatter(alldata_res.host_response_rate_num,alldata_res.y)

In [ ]:
alldata_res.info()

###カテゴリ変数への変換

In [ ]:
alldata_res
alldata_ctg=pd.get_dummies(alldata_res, drop_first=True)
alldata_ctg

###train testに分ける

In [ ]:
train=alldata_ctg.loc[alldata_ctg.TrainFlag]
train=train.drop("TrainFlag",axis=1)
train.head()

In [ ]:
train.columns

In [ ]:
test=alldata_ctg.loc[~alldata_ctg.TrainFlag]
test=test[test.columns[1:]]
test=test.drop("TrainFlag",axis=1)
test.head()

In [ ]:
test.shape

#相関関係

In [ ]:
train.corr().style.background_gradient("summer_r")

In [ ]:
train.corr()

#Predict

##XGBoost

In [ ]:
def xgb_train_cv(X_train_cv, y_train_cv, X_eval_cv, y_eval_cv, loop_counts):
    # データを格納する
    # 学習用
    xgb_train = xgb.DMatrix(X_train_cv, label=y_train_cv)
    # 検証用
    xgb_eval = xgb.DMatrix(X_eval_cv, label=y_eval_cv)
    # テスト用
    #xgb_test = xgb.DMatrix(X_test, label=y_test)

    # xgb_params = {
    #     'objective': ''loss_function': 'RMSE',  # 多値分類問題
    #     'num_class': 3,                 # 目的変数のクラス数
    #     'learning_rate': 0.1,           # 学習率
    #     'eval_metric': 'mlogloss'       # 学習用の指標 (Multiclass logloss)
    # }
    xgb_params = {
        "silent": 0,
        "max_depth": 50,
        "min_child_weight": 1,
        "eta": 0.2,
        "tree_method": "exact",
        "objective": 'reg:squarederror',
        "eval_metric": "rmse",
        # "predictor": "cpu_predictor" 
        'random_state':1234,
        }

    # 学習
    evals = [(xgb_train, 'train'), (xgb_eval, 'eval')] # 学習に用いる検証用データ
    evaluation_results = {}                            # 学習の経過を保存する箱
    bst = xgb.train(xgb_params,                        # 上記で設定したパラメータ
                    xgb_train,                         # 使用するデータセット
                    num_boost_round=1000,               # 学習の回数
                    early_stopping_rounds=5,          # アーリーストッピング
                    evals=evals,                       # 学習経過で表示する名称
                    evals_result=evaluation_results,   # 上記で設定した検証用データ
                    verbose_eval=0                     # 学習の経過の表示(非表示)
                    )
    
    # 検証用データで予測
    y_pred = bst.predict(xgb_eval, ntree_limit=bst.best_ntree_limit)
    # y_pred_max = np.argmax(y_pred, axis=1)

    print('Trial: ' + str(loop_counts))
    
    # # Accuracy の計算
    # accuracy = accuracy_score(y_eval_cv, y_pred_max)
    # print('XGBoost Accuracy:', accuracy)

    # plt.plot(evaluation_results["train"]["rmse"], color = "red", label = "train")
    # plt.plot(evaluation_results["eval"]["rmse"], color = "blue", label = "eval")
    # plt.legend()
    # plt.show()

    rmse_result=np.sqrt(mean_squared_error(y_eval_cv, y_pred))
    print('XGBoost RMSE:', rmse_result)
    
    return(bst, rmse_result,y_pred)

##LightGBM

In [ ]:
def lgbm_train_cv(X_train_cv, y_train_cv, X_eval_cv, y_eval_cv):
    # データを格納する
    # 学習用
    lgb_train = lgb.Dataset(X_train_cv, y_train_cv,
                            free_raw_data=False)
    # 検証用
    lgb_eval = lgb.Dataset(X_eval_cv, y_eval_cv, reference=lgb_train,
                           free_raw_data=False)
    
    # パラメータを設定
    params = {'task': 'train',                # レーニング ⇔　予測predict
              'boosting_type': 'gbdt',        # 勾配ブースティング
              # 'objective': 'multiclass',      # 目的関数：多値分類、マルチクラス分類
              'objective': 'regression',
              'metric': 'rmse',      # 検証用データセットで、分類モデルの性能を測る指標
              # 'num_class': 3,                 # 目的変数のクラス数
              'learning_rate': 0.1,           # 学習率（初期値0.1）
              'num_leaves': 40,               # 決定木の複雑度を調整（初期値31）
              'min_data_in_leaf': 1,          # データの最小数（初期値20）
             }

    # 学習
    evaluation_results = {}                                # 学習の経過を保存する箱
    LGBM_model = lgb.train(params,                              # 上記で設定したパラメータ
                      lgb_train,                           # 使用するデータセット
                      num_boost_round=1000,                 # 学習の回数
                      valid_names=['train', 'valid'],      # 学習経過で表示する名称
                      valid_sets=[lgb_train, lgb_eval],    # モデルの検証に使用するデータセット
                      evals_result=evaluation_results,     # 学習の経過を保存
                      early_stopping_rounds=5,            # アーリーストッピングの回数
                      verbose_eval=0)                      # 学習の経過を表示する刻み（非表示）

    # 検証用データで予測
    y_pred = LGBM_model.predict(X_eval_cv, num_iteration=LGBM_model.best_iteration)
    y_pred_max = np.argmax(y_pred)

    # # Accuracy の計算
    # accuracy = accuracy_score(y_eval_cv, y_pred_max)
    # print('LightGBM Accuracy:', accuracy)
    
    rmse_result=np.sqrt(mean_squared_error(y_eval_cv, y_pred))
    print('LightGBM RMSE:', rmse_result)
    
    return(LGBM_model, rmse_result,y_pred)

##CatBoost

In [ ]:
def catboost_train_cv(X_train_cv, y_train_cv, X_eval_cv, y_eval_cv):
    # データを格納する
    # 学習用
    CatBoost_train = Pool(X_train_cv, label=y_train_cv)
    # 検証用
    CatBoost_eval = Pool(X_eval_cv, label=y_eval_cv)

    # パラメータを設定
    params = {        
        'loss_function': 'RMSE',    # 多値分類問題
        'num_boost_round': 1000,          # 学習の回数
        'early_stopping_rounds': 5       # アーリーストッピングの回数
    }

    # 学習
    catb = CatBoost(params)
    catb.fit(CatBoost_train, eval_set=[CatBoost_eval], verbose=False)

    # 検証用データで予測
    y_pred = catb.predict(X_eval_cv)
    # y_pred_max = np.argmax(y_pred, axis=1)

    # # Accuracy の計算
    # accuracy = sum(y_eval_cv == y_pred_max) / len(y_eval_cv)
    # print('CatBoost Accuracy:', accuracy)

    rmse_result=np.sqrt(mean_squared_error(y_eval_cv, y_pred))
    print('CatBoost RMSE:', rmse_result)
    
    return(catb, rmse_result,y_pred)

##1段目

In [ ]:
# 各5つのモデルを保存するリストの初期化
xgb_models = []
lgbm_models = []
catb_models = []
# 各5つのモデルの正答率を保存するリストの初期化
xgb_rmse_result = []
lgbm_rmse_result = []
catb_rmse_result = []
# 学習のカウンター
loop_counts = 1

# X=train[train.columns[1:]].values
# Y=train[train.columns[0]].values
df_X=train[train.columns[1:]]
y= np.log(train[train.columns[0]])

# # 各クラスの確率（3モデル*5seed*1クラス）
first_preds = pd.DataFrame(np.zeros((len(df_X), 3*5)))


for seed_no in range(5): 
        
    # 学習データの数だけの数列（0行から最終行まで連番）
    row_no_list = list(range(len(df_X)))

    # KFoldクラスをインスタンス化（これを使って5分割する）Stratified Kfoldは、分類問題にしか使えません
    K_fold = StratifiedKFold(n_splits=5, shuffle=True,  random_state=42)
    #＃回帰問題でも＃回帰問題でもStratifiedKFoldが使えるように以下追記
    num_bins = int(np.floor(1+np.log2(len(y))))
    y_bin = pd.cut(y, bins=num_bins, labels=False)
    

    # KFoldクラスで分割した回数だけ実行（ここでは5回）
    for train_cv_no, eval_cv_no in K_fold.split(row_no_list, y_bin):
    # for train_cv_no, eval_cv_no in K_fold.split(row_no_list, y):
        # ilocで取り出す行を指定
        X_train_cv = df_X.iloc[train_cv_no, :]
        y_train_cv = pd.Series(y).iloc[train_cv_no]
        X_eval_cv = df_X.iloc[eval_cv_no, :]
        y_eval_cv = pd.Series(y).iloc[eval_cv_no]
        
        # XGBoostの訓練を実行
        bst, bst_rmse, bst_pred = xgb_train_cv(X_train_cv, y_train_cv, X_eval_cv, y_eval_cv, loop_counts)
        # LIghtGBMの訓練を実行
        LGBM_model, model_rmse, model_pred = lgbm_train_cv(X_train_cv, y_train_cv,X_eval_cv, y_eval_cv)
        # CatBoostの訓練を実行
        catb, catb_rmse, catb_pred = catboost_train_cv(X_train_cv, y_train_cv,X_eval_cv, y_eval_cv)
        # 実行回数のカウント
        loop_counts += 1
        
        # 学習が終わったモデルをリストに入れておく
        xgb_models.append(bst) 
        lgbm_models.append(LGBM_model) 
        catb_models.append(catb) 
        
        # 学習が終わったモデルの正答率をリストに入れておく
        xgb_rmse_result.append(bst_rmse) 
        lgbm_rmse_result.append(model_rmse) 
        catb_rmse_result.append(catb_rmse) 
        
        # # 検証データの各クラスの確率
        # for i in range(3):
        #     first_probs.iloc[eval_cv_no, (seed_no * 3) + i] = xgb_prob[:, i]
        #     first_probs.iloc[eval_cv_no, (seed_no * 3) + 15 + i] = lgbm_prob[:, i]
        #     first_probs.iloc[eval_cv_no, (seed_no * 3) + 30 + i] = catb_prob[:, i]

        # # 検証データの結果
        first_preds.iloc[eval_cv_no, (seed_no * 3) ] =bst_pred
        first_preds.iloc[eval_cv_no, (seed_no * 3) + 1] = model_pred
        first_preds.iloc[eval_cv_no, (seed_no * 3) + 2] = catb_pred

Trial: 1
XGBoost RMSE: 0.4042675942826259
LightGBM RMSE: 0.38255946115250605
CatBoost RMSE: 0.37912414883973333
Trial: 2
XGBoost RMSE: 0.4029719854791199
LightGBM RMSE: 0.38296729076376707
CatBoost RMSE: 0.3784810811239513
Trial: 3
XGBoost RMSE: 0.4018712399304369
LightGBM RMSE: 0.3774653570395306
CatBoost RMSE: 0.3739403564229324
Trial: 4
XGBoost RMSE: 0.4045914479573731
LightGBM RMSE: 0.3787927140985863
CatBoost RMSE: 0.3757628586926886
Trial: 5
XGBoost RMSE: 0.40669955032981997
LightGBM RMSE: 0.38255128676303407
CatBoost RMSE: 0.3795028539932154
Trial: 6
XGBoost RMSE: 0.4042675942826259
LightGBM RMSE: 0.38255946115250605
CatBoost RMSE: 0.37912414883973333
Trial: 7
XGBoost RMSE: 0.4029719854791199
LightGBM RMSE: 0.38296729076376707
CatBoost RMSE: 0.3784810811239513
Trial: 8
XGBoost RMSE: 0.4018712399304369
LightGBM RMSE: 0.3774653570395306
CatBoost RMSE: 0.3739403564229324
Trial: 9
XGBoost RMSE: 0.4045914479573731
LightGBM RMSE: 0.3787927140985863
CatBoost RMSE: 0.3757628586926886
Tr

In [ ]:
# y.max()

In [ ]:
first_preds

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
0,4.398381,4.375139,4.377673,4.398381,4.375139,4.377673,4.398381,4.375139,4.377673,4.398381,4.375139,4.377673,4.398381,4.375139,4.377673
1,4.187711,4.145050,4.120718,4.187711,4.145050,4.120718,4.187711,4.145050,4.120718,4.187711,4.145050,4.120718,4.187711,4.145050,4.120718
2,3.833019,4.107352,4.077034,3.833019,4.107352,4.077034,3.833019,4.107352,4.077034,3.833019,4.107352,4.077034,3.833019,4.107352,4.077034
3,4.733728,4.784438,4.778506,4.733728,4.784438,4.778506,4.733728,4.784438,4.778506,4.733728,4.784438,4.778506,4.733728,4.784438,4.778506
4,5.298479,5.273020,5.230051,5.298479,5.273020,5.230051,5.298479,5.273020,5.230051,5.298479,5.273020,5.230051,5.298479,5.273020,5.230051
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
55578,5.335371,5.461898,5.572010,5.335371,5.461898,5.572010,5.335371,5.461898,5.572010,5.335371,5.461898,5.572010,5.335371,5.461898,5.572010
55579,3.736951,3.942923,3.992293,3.736951,3.942923,3.992293,3.736951,3.942923,3.992293,3.736951,3.942923,3.992293,3.736951,3.942923,3.992293
55580,4.944480,4.719900,4.744932,4.944480,4.719900,4.744932,4.944480,4.719900,4.744932,4.944480,4.719900,4.744932,4.944480,4.719900,4.744932
55581,4.070905,3.893387,3.764673,4.070905,3.893387,3.764673,4.070905,3.893387,3.764673,4.070905,3.893387,3.764673,4.070905,3.893387,3.764673


In [ ]:
# 単独のモデルでの、テストデータの正答率
#スタッキングによる予測の１段目は、特徴量の抽出が目的なので、ここでの正答率はあまり気にしないでよいです。
print('XGBoost RMSE: ', np.array(xgb_rmse_result).mean())
print('LightGBM RMSE: ', np.array(lgbm_rmse_result).mean())
print('CatBoost RMSE: ', np.array(catb_rmse_result).mean())

XGBoost RMSE:  0.4040803635958752
LightGBM RMSE:  0.3808672219634848
CatBoost RMSE:  0.3773622598145042


###スタッキングによる予測

In [ ]:
loop_counts = 0

# 学習データとテストデータに分ける
X_train, X_test, y_train, y_test = train_test_split(first_preds, y.values,
                                                    test_size=0.2,
                                                    random_state=0)

# 予測結果の格納用のnumpy行列を作成
test_preds = np.zeros((len(y_test), 5))

# 学習データの数だけの数列（0行から最終行まで連番）
row_no_list = list(range(len(y_train)))

# KFoldクラスをインスタンス化（これを使って5分割する）
K_fold = StratifiedKFold(n_splits=5, shuffle=True,  random_state=0)
#＃回帰問題でも＃回帰問題でもStratifiedKFoldが使えるように以下追記
num_bins2 = int(np.floor(1+np.log2(len(y_train))))
y_train_bin = pd.cut(y_train, bins=num_bins2, labels=False)

# KFoldクラスで分割した回数だけ実行（ここでは5回）
for train_cv_no, eval_cv_no in K_fold.split(row_no_list, y_train_bin):
# for train_cv_no, eval_cv_no in K_fold.split(row_no_list, y_train):
    # ilocで取り出す行を指定
    X_train_cv = X_train.iloc[train_cv_no, :]
    y_train_cv = pd.Series(y_train).iloc[train_cv_no]
    X_eval_cv = X_train.iloc[eval_cv_no, :]
    y_eval_cv = pd.Series(y_train).iloc[eval_cv_no]

    # データを格納する
    # 学習用
    xgb_train = xgb.DMatrix(X_train_cv, label=y_train_cv)
    # 検証用
    xgb_eval = xgb.DMatrix(X_eval_cv, label=y_eval_cv)
    # テスト用
    xgb_test = xgb.DMatrix(X_test, label=y_test)

    # xgb_params = {
    #     'objective': 'multi:softprob',  # 多値分類問題
    #     'num_class': 3,                 # 目的変数のクラス数
    #     'learning_rate': 0.1,           # 学習率
    #     'eval_metric': 'mlogloss'       # 学習用の指標 (Multiclass logloss)
    # }
    xgb_params2 = {
        "silent": 0,
        "max_depth": 10,
        "min_child_weight": 1,
        "eta": 0.1,
        "tree_method": "exact",
        "objective": 'reg:squarederror',
        "eval_metric": "rmse",
        # "predictor": "cpu_predictor" 
        'random_state':1234,
        }

    # 学習
    evals = [(xgb_train, 'train'), (xgb_eval, 'eval')] # 学習に用いる検証用データ
    evaluation_results = {}                            # 学習の経過を保存する箱
    bst2 = xgb.train(xgb_params2,                        # 上記で設定したパラメータ
                    xgb_train,                         # 使用するデータセット
                    num_boost_round=1000,               # 学習の回数
                    early_stopping_rounds=5,          # アーリーストッピング
                    evals=evals,                       # 学習経過で表示する名称
                    evals_result=evaluation_results,   # 上記で設定した検証用データ
                    verbose_eval=0                     # 学習の経過の表示(非表示)
                    )


    y_pred = bst2.predict(xgb_test, ntree_limit=bst2.best_ntree_limit)
    # y_pred_max = np.argmax(y_pred, axis=1)
    
    # testの予測を保存
    test_preds[:, loop_counts] = y_pred
 
    print('Trial: ' + str(loop_counts))
    loop_counts += 1
    rmse__ = np.sqrt(mean_squared_error(y_test, y_pred))
    print('RMSE:', rmse__)

Trial: 0
RMSE: 0.38046787071503374
Trial: 1
RMSE: 0.3815074433997046
Trial: 2
RMSE: 0.3799573772229394
Trial: 3
RMSE: 0.3804769279511858
Trial: 4
RMSE: 0.3802720756760786


In [ ]:
import optuna

#学習データを説明変数と目的変数に分ける
# X=df_co_train[df_co_train.columns[1:]].values
# Y=df_co_train[df_co_train.columns[0]].values
X=train[train.columns[1:]].values
Y=train[train.columns[0]].values

# データを訓練データと正解データに7:3で分ける
X_train, X_test, y_train, y_test = train_test_split(first_preds, y.values,
                                                    test_size=0.2,
                                                    random_state=0)

#データをXGB専用のデータ型に変換
dtrain = xgb.DMatrix(X_train, label=y_train)
dtest = xgb.DMatrix(X_test, label=y_test)

def objective(trial):
  # 探索したいパラメータの範囲を記述 例えば、trial.suggest_int(“max_depth”, 6, 9)ならmax_depthを6,7,8,9の中で動かしながら探索
  params = {
        "silent": 0,
        "max_depth": trial.suggest_int("max_depth", 6, 10),
        "min_child_weight": 1,
        "eta": trial.suggest_loguniform("eta", 0.01, 0.3),
        "tree_method": "exact",
        "objective": 'reg:squarederror',
        "eval_metric": "rmse",
        # "predictor": "cpu_predictor" 
        'random_state':1234,  
    }
    # モデル定義、学習
  cv_results = xgb.cv(
      params,
      dtrain,
      num_boost_round=1000,
      seed=1234,
      nfold=5, # CVの分割数
      metrics={"rmse"},
      early_stopping_rounds=10
      )
    # 評価

  return cv_results["test-rmse-mean"].min()

study=optuna.create_study()
study.optimize(objective,n_trials=10)

[I 2022-01-14 13:28:12,638] A new study created in memory with name: no-name-32a1a3eb-9480-4553-ae2c-933ffccbdac1
[I 2022-01-14 13:28:32,496] Trial 0 finished with value: 0.3785738 and parameters: {'max_depth': 8, 'eta': 0.17180972975547049}. Best is trial 0 with value: 0.3785738.
[I 2022-01-14 13:30:52,053] Trial 1 finished with value: 0.3791174 and parameters: {'max_depth': 10, 'eta': 0.024621027569351448}. Best is trial 0 with value: 0.3785738.
[I 2022-01-14 13:31:16,362] Trial 2 finished with value: 0.3773526 and parameters: {'max_depth': 6, 'eta': 0.12136560143375674}. Best is trial 2 with value: 0.3773526.
[I 2022-01-14 13:33:40,823] Trial 3 finished with value: 0.37665740000000003 and parameters: {'max_depth': 6, 'eta': 0.016773482341374782}. Best is trial 3 with value: 0.37665740000000003.
[I 2022-01-14 13:36:22,138] Trial 4 finished with value: 0.3777368 and parameters: {'max_depth': 8, 'eta': 0.018733631105368395}. Best is trial 3 with value: 0.37665740000000003.
[I 2022-01-1

In [ ]:
print("Number of finished trials: ", len(study.trials))
print("Best trial:")
trial = study.best_trial

print("  Value: {}".format(trial.value))
print("  Params: ")
for key, value in trial.params.items():
    print("    {}: {}".format(key, value))

Number of finished trials:  10
Best trial:
  Value: 0.37665740000000003
  Params: 
    max_depth: 6
    eta: 0.016773482341374782


In [ ]:
#訓練データはdtrain、評価用のテストデータはdvalidと設定
watchlist = [(dtrain, 'train'), (dtest, 'eval')]
params = {
      "silent": 0,
      "max_depth": 50,
      "min_child_weight": 1,
      "eta": 0.2,
      "tree_method": "exact",
      "objective": 'reg:squarederror',
      "eval_metric": "rmse",
      # "predictor": "cpu_predictor" 
      'random_state':1234,  
  }
params["max_depth"] = trial.params["max_depth"]
params["eta"] = trial.params["eta"]
results_dict = {}
XGBmodel2 = xgb.train(params=params,
                  dtrain=dtrain,
                  num_boost_round=10000,
                  early_stopping_rounds=10,
                  evals=watchlist,
                  evals_result = results_dict,
                  )
plt.plot(results_dict["train"]["rmse"], color = "red", label = "train")
plt.plot(results_dict["eval"]["rmse"], color = "blue", label = "eval")
plt.legend()
plt.show()

##提出

In [ ]:
X_train

In [ ]:
test

In [ ]:
test.info()

In [ ]:
test_x_xgbst1=xgb.DMatrix(test)
lgb_test = lgb.Dataset(test, free_raw_data=False)
CatBoost_test = Pool(test)

test_first_preds = pd.DataFrame(np.zeros((len(test), 3*5)))#モデル数＊＊seed数

for seed in range(5):
  xgbst1 = bst.predict(test_x_xgbst1, ntree_limit=bst.best_ntree_limit)
  LGBM1=LGBM_model.predict(test, num_iteration=LGBM_model.best_iteration)
  catb1 = catb.predict(CatBoost_test)
  # # 検証データの結果
  test_first_preds.iloc[:, (seed * 3) ] =xgbst1
  test_first_preds.iloc[:, (seed * 3) + 1] = LGBM1
  test_first_preds.iloc[:, (seed * 3) + 2] = catb1


In [ ]:
#2段

# bst2_predict=bst2.predict(xgb.DMatrix(test_first_preds), ntree_limit=bst2.best_ntree_limit)
bst2_predict = XGBmodel2.predict(xgb.DMatrix(test_first_preds), ntree_limit=XGBmodel2.best_ntree_limit)
bst2_predict
len(bst2_predict)
# # kaggleの課題として PassengerId 0-18527 を予測することが要求されているため
# # idを生成しておく
# id = np.arange(18528)

# # Idと予測結果をデータフレーム化
# test_result = pd.DataFrame( { 'Id': id , 'SalePrice': np.exp(bst2_predict)} )
# print(test_result.head(10))

In [ ]:
# 生成したデータフレームをcsvとして保存
# test_result.to_csv( '/content/drive/MyDrive/Colab Notebooks/SIGNATE/minpaku-price/results/210113-1.csv' , header=False, index=False )
# test_result.to_csv( '/content/drive/MyDrive/Colab Notebooks/SIGNATE/minpaku-price/results/210113-1.csv' , index=False )